In [59]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import numpy as np
import pandas as pd
import h5py
from pathlib import Path

from sklearn.linear_model import Ridge,Lasso
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import flammkuchen as fl

from scipy.interpolate import interp1d

In [61]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from cycler import cycler

from megabouts.tracking_data import TrackingConfig, FullTrackingData, load_example_data
from megabouts.pipeline import FullTrackingPipeline
from megabouts.utils import (
    bouts_category_name,
    bouts_category_name_short,
    bouts_category_color,
    cmp_bouts,
)

In [ ]:
from mb_helper import compute_angle_between_vect_tail, compute_angle_between_vect, exptrapolate_segments, mid_head, midpoint
from mb_helper import tail_angles, fin_preprocess, calculate_angles

In [66]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [67]:
from megabouts_helper import labels_cat, color

In [68]:
# bouts_category_name_short, bouts_category_name

# Load Bouts

In [69]:
master_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Raw_Data')

fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths, len(fish_paths)


([WindowsPath('//portulab.synology.me/data/Kata/testdata/Raw_Data/240423_f0')],
 1)

In [70]:
fish= 0
fish_path = fish_paths[fish]
fish_id =  fish_paths[fish].name#[:-13]
exp_name = Path(fish_paths[fish]).parts[-2]
exp_name = 'testfish'
exp_name, fish_id

('testfish', '240423_f0')

In [71]:
out_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Processed_Data')

### Load DLC

In [72]:
### Load and replace eye data
#t, left eye [0], right eye [1], endpoints of eye vector end points (point1xy, point2xy) for anterior and posterior
eye_coords = fl.load(fish_path/'eye_coords.h5')['eye_coords']
eye_coords.shape

(95423, 2, 2, 2)

In [73]:
left_eye_points = np.asarray(eye_coords)[:,0]
right_eye_points = np.asarray(eye_coords)[:,1]

l_anterior = np.asarray(left_eye_points[:,0])
l_posterior = np.asarray(left_eye_points[:,1])

left_eye_anterior_x = []
left_eye_anterior_y = []
left_eye_posterior_x = []
left_eye_posterior_y = []
for i in range(l_anterior.shape[0]):
    left_eye_anterior_x.append(l_anterior[i][0])
    left_eye_anterior_y.append(l_anterior[i][1])
    left_eye_posterior_x.append(l_posterior[i][0])
    left_eye_posterior_y.append(l_posterior[i][1])
    
r_anterior = np.asarray(right_eye_points[:,0])
r_posterior = np.asarray(right_eye_points[:,1])

right_eye_anterior_x = []
right_eye_anterior_y = []
right_eye_posterior_x = []
right_eye_posterior_y = []
for i in range(r_anterior.shape[0]):
    right_eye_anterior_x.append(r_anterior[i][0])
    right_eye_anterior_y.append(r_anterior[i][1])
    right_eye_posterior_x.append(r_posterior[i][0])
    right_eye_posterior_y.append(r_posterior[i][1])
    

## Load DLC

In [74]:
fps=200
mm_per_unit = 1/70
N_seg = 10
N = eye_coords.shape[0]

In [75]:
vid_path = list(fish_path.glob('*video*'))[0]
filename = list(fish_path.glob('*316000.h5*'))[0]
df= pd.read_hdf(filename,  header=[1, 2], index_col=0)
df = df['DLC_resnet50_dlc_2Dec12shuffle1_316000']
print(f'{df.shape[0]/(fps*60)} minutes at {fps} fps')
print('working on {} frames'.format(df.shape[0]))

#Extract angles
body_x = df.body.values[:, 0].astype('float')
body_y = df.body.values[:, 1].astype('float')

tail_x_col = [f'tail_{i}' for i in range(5)]
tail_y_col = [f'tail_{i}' for i in range(5)]
tail_x = np.array([df[x].iloc[:, 0].values.astype('float') for x in tail_x_col])
tail_y = np.array([df[x].iloc[:, 1].values.astype('float') for x in tail_y_col])

# upsample tail to 10 segments
tail_x_10, tail_y_10 = exptrapolate_segments(tail_x, tail_y, N_seg)


7.9519166666666665 minutes at 200 fps
working on 95423 frames


In [76]:
tail_x_10.shape

(11, 95423)

In [77]:
tail_x_10[5,:]

array([545.1464873 , 549.63523213, 552.34921732, ..., 400.76044311,
       399.75724162, 398.42930308])

In [78]:
## replace eyes and recompute mid head

In [79]:
mid_headx, mid_heady, left_mid_eye_x, left_mid_eye_y, right_mid_eye_x, right_mid_eye_y = mid_head(
             np.asarray(right_eye_posterior_x), np.asarray(right_eye_posterior_y),
             np.asarray(right_eye_anterior_x), np.asarray(right_eye_anterior_y),
             np.asarray(left_eye_posterior_x), np.asarray(left_eye_posterior_y),
             np.asarray(left_eye_anterior_x), np.asarray(left_eye_anterior_y))


In [80]:
head_x = mid_headx * mm_per_unit
head_y = mid_heady * mm_per_unit

In [81]:
head_x.shape, head_y.shape

((95423,), (95423,))

#### Add head to df

In [82]:
# Now, let's add 'head_x' and 'head_y' keypoints under 'head' with the same likelihood as 'body'
for i in range(11):
    df[('tail_{}'.format(i), 'x')] = tail_x_10[i,:] 
    df[('tail_{}'.format(i), 'y')] = tail_y_10[i,:]
    df[('tail_{}'.format(i), 'likelihood')] = df[('body', 'likelihood')] # Copy likelihood of body to head

In [83]:
df[('left_eye_anterior', 'x')] = left_eye_anterior_x 
df[('left_eye_anterior', 'y')] = left_eye_anterior_y 

df[('left_eye_posterior', 'x')] = left_eye_posterior_x 
df[('left_eye_posterior', 'y')] = left_eye_posterior_y 

df[('right_eye_anterior', 'x')] = right_eye_anterior_x 
df[('right_eye_anterior', 'y')] = right_eye_anterior_y 

df[('right_eye_posterior', 'x')] = right_eye_posterior_x 
df[('right_eye_posterior', 'y')] = right_eye_posterior_y 

In [84]:
# Now, let's add 'head_x' and 'head_y' keypoints under 'head' with the same likelihood as 'body'
df[('mid_head', 'x')] = mid_headx 
df[('mid_head', 'y')] = mid_heady 
df[('mid_head', 'likelihood')] = df[('body', 'likelihood')] # Copy likelihood of body to head


In [85]:
df.head()

bodyparts left_eye_anterior                        left_eye_posterior  \
coords                    x           y likelihood                  x   
0                645.326900  521.443017   0.999891         654.856002   
1                646.183150  519.644351   0.999896         656.078755   
2                650.207855  517.061760   0.999923         659.556388   
3                654.625610  513.145553   0.999913         664.477157   
4                657.924704  510.390997   0.999740         667.236973   

bodyparts                        right_eye_anterior                         \
coords              y likelihood                  x           y likelihood   
0          553.228951   0.999758         606.735142  539.762843   0.999914   
1          551.395331   0.999851         607.531952  537.669198   0.999881   
2          549.311521   0.999905         611.376284  534.990023   0.999691   
3          545.056029   0.999953         616.157773  530.418686   0.999719   
4          542.307606   0.999960         619.216163  528.236603   0.999846   

bodyparts right_eye_posterior  ...     tail_8      tail_9              \
coords                      x  ... likelihood           x           y   
0                  628.310659  ...   0.998467  463.398844  250.457989   
1                  629.298588  ...   0.998625  473.571332  246.601053   
2                  632.566573  ...   0.999190  491.481593  238.960160   
3                  637.127398  ...   0.997801  503.204287  227.416002   
4                  640.460880  ...   0.998626  506.768862  217.844257   

bodyparts                tail_10                           mid_head  \
coords    likelihood           x           y likelihood           x   
0           0.998467  439.471039  239.365433   0.998467  633.807176   
1           0.998625  444.348114  245.641525   0.998625  634.773111   
2           0.999190  461.540009  236.261642   0.999190  638.426775   
3           0.997801  476.081390  216.947006   0.997801  643.096984   
4           0.998626  485.563690  199.774689   0.998626  646.209680   

bodyparts                         
coords              y likelihood  
0          544.905587   0.998467  
1          542.948934   0.998625  
2          540.509511   0.999190  
3          536.308950   0.997801  
4          533.732044   0.998626  

[5 rows x 63 columns]

In [86]:
df.to_csv(master_path/ 'DLC_mod.csv')


In [87]:
df_dlc = pd.read_csv(master_path/ 'DLC_mod.csv', header=[0, 1])


In [88]:
df_dlc.head()

bodyparts left_eye_anterior                        left_eye_posterior  \
     coords                 x           y likelihood                  x   
0         0        645.326900  521.443017   0.999891         654.856002   
1         1        646.183150  519.644351   0.999896         656.078755   
2         2        650.207855  517.061760   0.999923         659.556388   
3         3        654.625610  513.145553   0.999913         664.477157   
4         4        657.924704  510.390997   0.999740         667.236973   

                         right_eye_anterior                         ...  \
            y likelihood                  x           y likelihood  ...   
0  553.228951   0.999758         606.735142  539.762843   0.999914  ...   
1  551.395331   0.999851         607.531952  537.669198   0.999881  ...   
2  549.311521   0.999905         611.376284  534.990023   0.999691  ...   
3  545.056029   0.999953         616.157773  530.418686   0.999719  ...   
4  542.307606   0.999960         619.216163  528.236603   0.999846  ...   

      tail_8      tail_9                            tail_10              \
  likelihood           x           y likelihood           x           y   
0   0.998467  463.398844  250.457989   0.998467  439.471039  239.365433   
1   0.998625  473.571332  246.601053   0.998625  444.348114  245.641525   
2   0.999190  491.481593  238.960160   0.999190  461.540009  236.261642   
3   0.997801  503.204287  227.416002   0.997801  476.081390  216.947006   
4   0.998626  506.768862  217.844257   0.998626  485.563690  199.774689   

                mid_head                         
  likelihood           x           y likelihood  
0   0.998467  633.807176  544.905587   0.998467  
1   0.998625  634.773111  542.948934   0.998625  
2   0.999190  638.426775  540.509511   0.999190  
3   0.997801  643.096984  536.308950   0.997801  
4   0.998626  646.209680  533.732044   0.998626  

[5 rows x 64 columns]